# MEPs in diseased vs healthy mice

In [1]:
from tqdm import tqdm
from mepextract.extracting import Extractor
from mepextract.extracting import Viewer

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
import os
import pickle
import json

plt.style.use(['science', 'grid', 'vibrant'])

## defining relevant folders and paths

In [ ]:
test = '/Users/fomo/Documents/Research/UNIC Research/Motor Evoked Potentials Test/MICE_EEG_2024 (93-102).csv'
full = '/Users/fomo/Documents/Research/UNIC Research/Motor Evoked Potentials Test/MICE_EEG_2024 (93-257).csv'

spreadsheet = pd.read_csv(full)

master_folder = '/Volumes/STORAGE 1.0/UNIC Research/Motor Evoked Potentials/MEPSSEP_raw_data/post 93'
test_folder = '/Users/fomo/Documents/Research/UNIC Research/Motor Evoked Potentials Test'

spreadsheet

## extracting MEP matrices for all trials

In [ ]:
# list to store each trial
sampling_rate = 30000

all_extracted = []

n = len(spreadsheet)
for i in tqdm(range(0, n), desc="Processing trials"):
    if spreadsheet['sessionType'][i] == 'reject':
        continue
    else:
        # relevant information
        notes = spreadsheet.loc[i]
        
        # defining extractor object
        extractor = Extractor(
            master_folder=master_folder, 
            notes=notes,
            recording_channels=[5, 7],
            sampling_rate=sampling_rate,
            pre_stimulus_ms=10,
            post_stimulus_ms=100)
        
        # extracting relevant data
        extracted = extractor.lazy(event_channel=13, export=True)
    
        all_extracted.append(extracted)

In [ ]:
# saving extracted data
with open('/Users/fomo/Documents/Research/UNIC Research/Motor Evoked Potentials Test/extracted.pkl', 'wb') as f:
    pickle.dump(all_extracted, f)

In [2]:
# loading extracted data
extract_location = '/Users/fomo/Documents/Research/UNIC Research/Motor Evoked Potentials Test/extracted.pkl'

with open(extract_location, 'rb') as f:
    extracted = pickle.load(f)

## trial classification and peak detection

In [5]:
viewer = Viewer(extracted = extracted, sampling_rate=30000)

viewer.classifier()

Output()

HTML(value='<br>')

Output(layout=Layout(height='1000', width='95%'))

HTML(value='<br>')

In [16]:
accepted = viewer.accepted
unsure = viewer.unsure
rejected = viewer.rejected

accepted_indices = viewer.accepted_indices
unsure_indices = viewer.unsure_indices
rejected_indices = viewer.rejected_indices

classified_indices = {'accepted': accepted_indices, 'unsure': unsure_indices, 'rejected': rejected_indices}

with open('/Users/fomo/Documents/Research/UNIC Research/Motor Evoked Potentials Test/accepted.pkl', 'wb') as f:
    pickle.dump(accepted, f)

with open('/Users/fomo/Documents/Research/UNIC Research/Motor Evoked Potentials Test/unsure.pkl', 'wb') as f:
    pickle.dump(unsure, f)
    
with open('/Users/fomo/Documents/Research/UNIC Research/Motor Evoked Potentials Test/rejected.pkl', 'wb') as f:
    pickle.dump(rejected, f)
    
with open('/Users/fomo/Documents/Research/UNIC Research/Motor Evoked Potentials Test/classified_indices.pkl', 'wb') as f:
    pickle.dump(classified_indices, f)


In [ ]:
with open('/Users/fomo/Documents/Research/UNIC Research/Motor Evoked Potentials Test/accepted.pkl', 'rb') as f:
    accepted = pickle.load(f)

with open('/Users/fomo/Documents/Research/UNIC Research/Motor Evoked Potentials Test/unsure.pkl', 'rb') as f:
    unsure = pickle.load(f)

with open('/Users/fomo/Documents/Research/UNIC Research/Motor Evoked Potentials Test/unsure.pkl', 'rb') as f:
    classified_indices = pickle.load(f)

## peak selection (first detected peak)

In [ ]:
selected_peaks = []

verify = []

exclude = [44, 49, 50, 53, 55, 56, 57, 62, 63, 68, 73, 74, 80, 81, 86, 87, 92, 93, 94, 100, 101, 106, 107, 108, 109, 110, 111, 112, 113, 118, 123, 124, 125, 130, 131, 132, 134, 135, 137, 138, 140, 141, 142, 143]

for trial in detected_peaks:
    
    if trial['index'] not in exclude:
        
        group = trial['group']
        current = trial['current']
        index = trial['index']
        
        # sort peaks by amplitude
        largest_peaks = []
        positive_peaks = (sorted(trial['positive peaks'], key=lambda x: x[1]))
        negative_peaks = (sorted(trial['negative peaks'], key=lambda x: x[1]))
        
        if len(positive_peaks)>0:
            largest_peaks.append(positive_peaks[-1])
        if len(negative_peaks)>0:
            largest_peaks.append(negative_peaks[-1])
        
        # sort peaks by time
        largest_peaks = sorted(largest_peaks, key=lambda x: x[0])
        
        #first positive peak': positive_peaks[-1], 'first negative peak': negative_peaks[-1], 
        if len(largest_peaks) > 0:
            selected_peaks.append({'group': group, 'current': current, 'first_large_peak': largest_peaks[0]})
            if largest_peaks[0][0] <  345:
                verify.append(index)

seen = set()
verify = [item for item in verify if item not in seen and not seen.add(item)]

toVerify = []

for index, value in enumerate(verify):
    toVerify.append((extracted[value], value))

## plotting delays vs amplitude

In [ ]:
conversion = (1 / sampling_rate) * 1000

# Extract delays, amplitudes, and groups
delays = [peak['first_large_peak'][0]*conversion - 10 for peak in selected_peaks]
amplitudes = [peak['first_large_peak'][1] for peak in selected_peaks]
groups = [peak['group'] for peak in selected_peaks]
currents = [peak['current'] for peak in selected_peaks]

# min-max scaling current
from sklearn.preprocessing import MinMaxScaler

currents_2d = [[x] for x in currents] # 2-D array for MinMaxScaler() 
scaler = MinMaxScaler()
scaled_currents = scaler.fit_transform(currents_2d).flatten()

# separate data by group
group1_delays = [delays[i] for i in range(len(groups)) if groups[i] == 1]
group1_amplitudes = [amplitudes[i] for i in range(len(groups)) if groups[i] == 1]
group2_delays = [delays[i] for i in range(len(groups)) if groups[i] == 2]
group2_amplitudes = [amplitudes[i] for i in range(len(groups)) if groups[i] == 2]

# calculate means and standard deviations for delays and amplitudes for each group
mean_delay_1, std_delay_1 = np.mean(group1_delays), np.std(group1_delays)
mean_amplitude_1, std_amplitude_1 = np.mean(group1_amplitudes), np.std(group1_amplitudes)

mean_delay_2, std_delay_2 = np.mean(group2_delays), np.std(group2_delays)
mean_amplitude_2, std_amplitude_2 = np.mean(group2_amplitudes), np.std(group2_amplitudes)

# plotting
from matplotlib.colors import ListedColormap, colorConverter

plt.figure(figsize=(14, 7))
plt.scatter(delays, amplitudes, c=groups, cmap=ListedColormap( ['green', 'red']), alpha=scaled_currents)

# group 1 with error bars
plt.errorbar(mean_delay_1, mean_amplitude_1, xerr=std_delay_1, yerr=std_amplitude_1, fmt='o', color='green', label='Group 1', capsize=5)

# group 2 with error bars
plt.errorbar(mean_delay_2, mean_amplitude_2, xerr=std_delay_2, yerr=std_amplitude_2, fmt='o', color='red', label='Group 2', capsize=5)

# Adding labels and title
plt.xlabel('Delay (ms)', fontsize=14)
plt.ylabel('Amplitude (SDs)', fontsize=14)
plt.title('Delays vs Amplitudes with Standard Deviation Error Bars', fontsize=17)
plt.legend()
plt.savefig('/Users/fomo/Desktop/delays_vs_amplitudes.png')
plt.show()

## statistical tests

In [ ]:
from scipy.stats import ttest_ind, mannwhitneyu, shapiro

shapiro_group1 = shapiro(group1_delays)
shapiro_group2 = shapiro(group2_delays)

print("Shapiro-Wilk test for Group 1 Delays: p-value =", shapiro_group1.pvalue)
print("Shapiro-Wilk test for Group 2 Delays: p-value =", shapiro_group2.pvalue)

# Determine if the data is normally distributed based on p-value > 0.05
if shapiro_group1.pvalue > 0.05 and shapiro_group2.pvalue > 0.05:
    # If data are approximately normal, use independent samples t-test
    t_stat, p_value = ttest_ind(group1_delays, group2_delays, equal_var=False)  # Welch's t-test
    print("Independent t-test p-value:", p_value)
else:
    # If data are not normally distributed, use Mann-Whitney U test
    u_stat, p_value = mannwhitneyu(group1_delays, group2_delays, alternative='two-sided')
    print("Mann-Whitney U test p-value:", p_value)

alpha = 0.05  
if p_value < alpha:
    print("The difference in delays between Group 1 and Group 2 is statistically significant.")
else:
    print("No statistically significant difference in delays between Group 1 and Group 2.")
